# Submission 1
#### For Python Level 3, Prepare Level 2, and Access Level 3
In this section of my portfolio I will be demonstrating skills for the above list. For more demonstration on **Prepare**, please look at *submission_4*. 

##### Access Level 3
Access data from both common and uncommon formats and identify best practices for formats in different contexts. 
<p>
There are many different ways to access data in Pandas, whether you want to read into a more common format such as a CSV file, or if you to access data stored in an HDF5 table or in SAS format. 
</p>
<p> 
When considering which format to use, you should also note the different types of separators that are the defaults of each method. For example, pd.read_csv defaults separation to a comma, while pd.read_table defaults to'\t'. Different file types also come with different parameters (whether optional or required) according to their designated Pandas function. 
</p>
<p>
For a contextual example, consider pd.read_sql_table, pd.read_sql_query, and pd.read_sql. Although these functions seem very similar to each other, there are actually quite a few key differences. The list below describes each method and how you can use them.
</p>

**pd.read_sql_table** 
* Reads an SQL database table into a DataFrame
* Takes in parse_dates, columns, chunksize
* Does not support DBAPI connections

**pd.read_sql_query**
* Reads an SQL query into a DataFrame
* Takes in parse_dates, chunksize
* May support DBAPI connections depending on type

**pd.read_sql** 
* Read SQL query or database table into a DataFrame
* More for convenience, compatibility for previous methods
* Takes in parse_dates, columns, chunksize

Now, I will be using SQLite to access the database for a dataset called **Simple Folk**.
<p>
Link to database: http://2016.padjo.org/files/data/starterpack/simplefolks.sqlite

In [1]:
import sqlite3
import pandas as pd
con = sqlite3.connect('simplefolks.sqlite')
cur = con.cursor()
rows = cur.fetchall()

In [2]:
# DataFrame with the age, sex, and name of individuals
people_df = pd.read_sql_query("SELECT age, sex, name FROM people", con)
# DataFrame with pets, pet owners, and pet names
pet_df = pd.read_sql_query("SELECT name, owner_name FROM pets ORDER BY name", con)

In [3]:
# Viewing DataFrames
people_df.head()

,age,sex,name
0,33,M,Austin
1,90,M,Blair
2,28,F,Carolina
3,41,F,Dani
4,70,M,Donald


In [4]:
# Viewing DataFrames
pet_df.head()

,name,owner_name
0,Artemis,Dani
1,Bumpkin,Georgina
2,Donald,Donald
3,Essy,Sherry
4,Harambe,Dani


Now, let's say we wanted to list all of the 30 year old and older men in the people table. We can do this by implementing the query below.

In [5]:
query_1 = pd.read_sql_query("SELECT * FROM people WHERE sex = 'M' AND age >= 30", con)
query_1

,name,sex,age
0,Austin,M,33
1,Blair,M,90
2,Donald,M,70
3,Michael,M,48
4,Zed,M,42


Or, with the pet table we can find the pets name and type that are not dogs or cats. Personally, the bird named Harambe is my favorite.

In [6]:
query_2 = pd.read_sql_query("SELECT name, type FROM pets WHERE type != 'cat' and TYPE != 'dog'", con)
query_2

,name,type
0,Harambe,bird
1,Hodor,bird
2,Bumpkin,horse
3,Secretariat,horse
4,Harry Potter,horse
5,Xerses,horse
6,Jenkins,horse


Luckily for us, SQLite databases and their respective tables usually come in a very easy-to-read format. In other cases where the data is not as easy to read in, we need to be able to clean and prepare it.

##### Prepare Level 2
Apply data reshaping, cleaning, and filtering as directed.

In [7]:
# First, I'll load the xls file in and take a look at it uncleaned.
unclean_table = pd.read_excel("tabn039.xls")
unclean_table.head()

,"Table 39. Enrollment in public elementary and secondary schools, by level and grade: Selected years, fall 1980 through fall 2010",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,Grade,Fall 1980,Fall 1985,Fall 1990,Fall 1992,Fall 1993,Fall 1994,Fall 1995,Fall 1996,Fall 1997,...,Fall 2001,Fall 2002,Fall 2003,Fall 2004,Fall 2005,Fall 2006,Fall 2007,Fall 2008,Fall 2009,Fall 2010
1,1,2,3,4,5,6,7,8,9,10,...,14,15,16,17,18,19,20,21,22,23
2,NaN,Number (in thousands),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,All grades .................,40877.5,39422,41216.7,42823.3,43464.9,44111.5,44840.5,45611,46126.9,...,47671.9,48183.1,48540.2,48795.5,49113.3,49315.8,49292.5,49265.6,49361,49484.2
4,Elementary ............................,27646.5,27034.2,29875.9,31086,31501.8,31896.1,32338.3,32762.3,33071,...,33935.9,34114.2,34200.7,34177.6,34204,34234.8,34205.4,34285.6,34409.3,34624.5


Obviously, this table needs some major cleaning. Although I am only showing the head of the table above, there are actually plenty of rows towards the end (rows 42-45) that are filled with NaN values. Our first task will be to **drop and rename** columns and rows.

In [8]:
clean_table = pd.read_excel("tabn039.xls", header = 1, index_col=0)
clean_table.head()

,Fall 1980,Fall 1985,Fall 1990,Fall 1992,Fall 1993,Fall 1994,Fall 1995,Fall 1996,Fall 1997,Fall 1998,...,Fall 2001,Fall 2002,Fall 2003,Fall 2004,Fall 2005,Fall 2006,Fall 2007,Fall 2008,Fall 2009,Fall 2010
Grade,,,,,,,,,,,,,,,,,,,,,
1,2,3.000,4.000,5.000,6.000,7.000,8.000,9.000,10.000,11.000,...,14.000,15.000,16.000,17.000,18.000,19.000,20.000,21.000,22.000,23.000
NaN,Number (in thousands),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
All grades .................,40877.5,39421.961,41216.683,42823.312,43464.916,44111.482,44840.481,45611.046,46126.897,46538.585,...,47671.870,48183.086,48540.215,48795.465,49113.298,49315.842,49292.507,49265.572,49360.982,49484.181
Elementary ............................,27646.5,27034.244,29875.914,31086.039,31501.847,31896.076,32338.311,32762.256,33071.020,33343.733,...,33935.922,34114.245,34200.741,34177.565,34203.962,34234.751,34205.362,34285.564,34409.260,34624.530
Prekindergarten ........,96.133,151.335,303.319,504.585,545.135,602.535,636.846,670.373,695.120,728.703,...,865.086,914.879,949.649,990.421,1036.466,1084.391,1081.125,1179.507,1222.860,1278.678


In [9]:
# This will get rid of leadings rows, as well as NaN rows towards the bottom
clean_table = clean_table.iloc[2:41]
clean_table

,Fall 1980,Fall 1985,Fall 1990,Fall 1992,Fall 1993,Fall 1994,Fall 1995,Fall 1996,Fall 1997,Fall 1998,...,Fall 2001,Fall 2002,Fall 2003,Fall 2004,Fall 2005,Fall 2006,Fall 2007,Fall 2008,Fall 2009,Fall 2010
Grade,,,,,,,,,,,,,,,,,,,,,
All grades .................,40877.5,39421.961000,41216.683000,42823.312000,43464.916000,44111.482000,44840.481000,45611.046000,46126.897000,46538.585000,...,47671.870000,48183.086000,48540.215000,48795.465000,49113.298000,49315.842000,49292.507000,49265.572000,49360.982000,49484.181000
Elementary ............................,27646.5,27034.244000,29875.914000,31086.039000,31501.847000,31896.076000,32338.311000,32762.256000,33071.020000,33343.733000,...,33935.922000,34114.245000,34200.741000,34177.565000,34203.962000,34234.751000,34205.362000,34285.564000,34409.260000,34624.530000
Prekindergarten ........,96.133,151.335000,303.319000,504.585000,545.135000,602.535000,636.846000,670.373000,695.120000,728.703000,...,865.086000,914.879000,949.649000,990.421000,1036.466000,1084.391000,1081.125000,1179.507000,1222.860000,1278.678000
Kindergarten ........,2593.11,3041.071000,3306.329000,3312.777000,3376.998000,3444.276000,3536.227000,3532.263000,3503.491000,3442.874000,...,3379.151000,3433.895000,3503.280000,3543.554000,3619.421000,3631.002000,3609.415000,3639.750000,3677.971000,3682.092000
1st grade ......................,2894.47,3238.855000,3499.434000,3541.708000,3529.072000,3593.382000,3670.903000,3770.420000,3754.594000,3727.158000,...,3614.069000,3593.839000,3612.509000,3663.005000,3690.854000,3751.144000,3750.073000,3708.485000,3728.773000,3753.951000
2nd grade ......................,2799.59,2940.995000,3327.454000,3431.291000,3429.243000,3439.828000,3507.022000,3600.076000,3688.972000,3681.463000,...,3593.337000,3564.873000,3543.781000,3559.854000,3606.405000,3640.702000,3704.007000,3698.723000,3664.697000,3700.668000
3rd grade ......................,2893.01,2894.524000,3296.720000,3361.362000,3436.978000,3439.042000,3444.740000,3523.762000,3596.887000,3695.643000,...,3653.314000,3622.790000,3611.041000,3580.462000,3586.107000,3627.487000,3658.869000,3707.772000,3707.038000,3685.833000
4th grade ........................,3107.13,2771.015000,3248.065000,3341.737000,3361.110000,3425.951000,3430.583000,3453.769000,3507.457000,3592.153000,...,3694.898000,3668.517000,3619.089000,3611.638000,3577.506000,3585.891000,3624.408000,3646.896000,3700.530000,3711.341000
5th grade ..........................,3129.86,2776.402000,3197.343000,3325.427000,3350.453000,3371.884000,3437.943000,3452.853000,3457.541000,3520.104000,...,3726.835000,3711.466000,3684.539000,3635.181000,3632.829000,3601.796000,3600.120000,3629.079000,3652.134000,3717.521000


In [10]:
# Changing row and column names to appropriate names
clean_table.rename(str.lower, axis='index', inplace=True)
clean_table.rename(str.lower, axis='columns', inplace=True)
# Deleting whitespaces and trailing periods
clean_table.index = clean_table.index.str.replace('.','')
clean_table.index = clean_table.index.str.replace(' ','_')
clean_table.index = clean_table.index.str.lstrip('_')
clean_table.index = clean_table.index.str.rstrip('_')
clean_table.columns = clean_table.columns.str.replace(' ','_')

In [11]:
clean_table

,fall_1980,fall_1985,fall_1990,fall_1992,fall_1993,fall_1994,fall_1995,fall_1996,fall_1997,fall_1998,...,fall_2001,fall_2002,fall__2003,fall__2004,fall__2005,fall__2006,fall__2007,fall__2008,fall__2009,fall__2010
Grade,,,,,,,,,,,,,,,,,,,,,
all_grades,40877.5,39421.961000,41216.683000,42823.312000,43464.916000,44111.482000,44840.481000,45611.046000,46126.897000,46538.585000,...,47671.870000,48183.086000,48540.215000,48795.465000,49113.298000,49315.842000,49292.507000,49265.572000,49360.982000,49484.181000
elementary,27646.5,27034.244000,29875.914000,31086.039000,31501.847000,31896.076000,32338.311000,32762.256000,33071.020000,33343.733000,...,33935.922000,34114.245000,34200.741000,34177.565000,34203.962000,34234.751000,34205.362000,34285.564000,34409.260000,34624.530000
prekindergarten,96.133,151.335000,303.319000,504.585000,545.135000,602.535000,636.846000,670.373000,695.120000,728.703000,...,865.086000,914.879000,949.649000,990.421000,1036.466000,1084.391000,1081.125000,1179.507000,1222.860000,1278.678000
kindergarten,2593.11,3041.071000,3306.329000,3312.777000,3376.998000,3444.276000,3536.227000,3532.263000,3503.491000,3442.874000,...,3379.151000,3433.895000,3503.280000,3543.554000,3619.421000,3631.002000,3609.415000,3639.750000,3677.971000,3682.092000
1st_grade,2894.47,3238.855000,3499.434000,3541.708000,3529.072000,3593.382000,3670.903000,3770.420000,3754.594000,3727.158000,...,3614.069000,3593.839000,3612.509000,3663.005000,3690.854000,3751.144000,3750.073000,3708.485000,3728.773000,3753.951000
2nd_grade,2799.59,2940.995000,3327.454000,3431.291000,3429.243000,3439.828000,3507.022000,3600.076000,3688.972000,3681.463000,...,3593.337000,3564.873000,3543.781000,3559.854000,3606.405000,3640.702000,3704.007000,3698.723000,3664.697000,3700.668000
3rd_grade,2893.01,2894.524000,3296.720000,3361.362000,3436.978000,3439.042000,3444.740000,3523.762000,3596.887000,3695.643000,...,3653.314000,3622.790000,3611.041000,3580.462000,3586.107000,3627.487000,3658.869000,3707.772000,3707.038000,3685.833000
4th_grade,3107.13,2771.015000,3248.065000,3341.737000,3361.110000,3425.951000,3430.583000,3453.769000,3507.457000,3592.153000,...,3694.898000,3668.517000,3619.089000,3611.638000,3577.506000,3585.891000,3624.408000,3646.896000,3700.530000,3711.341000
5th_grade,3129.86,2776.402000,3197.343000,3325.427000,3350.453000,3371.884000,3437.943000,3452.853000,3457.541000,3520.104000,...,3726.835000,3711.466000,3684.539000,3635.181000,3632.829000,3601.796000,3600.120000,3629.079000,3652.134000,3717.521000


Now, our rows and columns look pretty good. However, we have a couple of different tables combined into one table (cleaned_table). Below, I will attempt to split these into **2 different tables**.

This table shows the **Enrollment in Schools, in Thousands**.

In [23]:
# First table
df1 = clean_table.iloc[:19]
df1.style.set_caption("Enrollment in Schools, in Thousands")
df1.index.name = None
df1

,fall_1980,fall_1985,fall_1990,fall_1992,fall_1993,fall_1994,fall_1995,fall_1996,fall_1997,fall_1998,...,fall_2001,fall_2002,fall__2003,fall__2004,fall__2005,fall__2006,fall__2007,fall__2008,fall__2009,fall__2010
all_grades,40877.5,39421.961,41216.683,42823.312,43464.916,44111.482,44840.481,45611.046,46126.897,46538.585,...,47671.870,48183.086,48540.215,48795.465,49113.298,49315.842,49292.507,49265.572,49360.982,49484.181
elementary,27646.5,27034.244,29875.914,31086.039,31501.847,31896.076,32338.311,32762.256,33071.020,33343.733,...,33935.922,34114.245,34200.741,34177.565,34203.962,34234.751,34205.362,34285.564,34409.260,34624.530
prekindergarten,96.133,151.335,303.319,504.585,545.135,602.535,636.846,670.373,695.120,728.703,...,865.086,914.879,949.649,990.421,1036.466,1084.391,1081.125,1179.507,1222.860,1278.678
kindergarten,2593.11,3041.071,3306.329,3312.777,3376.998,3444.276,3536.227,3532.263,3503.491,3442.874,...,3379.151,3433.895,3503.280,3543.554,3619.421,3631.002,3609.415,3639.750,3677.971,3682.092
1st_grade,2894.47,3238.855,3499.434,3541.708,3529.072,3593.382,3670.903,3770.420,3754.594,3727.158,...,3614.069,3593.839,3612.509,3663.005,3690.854,3751.144,3750.073,3708.485,3728.773,3753.951
2nd_grade,2799.59,2940.995,3327.454,3431.291,3429.243,3439.828,3507.022,3600.076,3688.972,3681.463,...,3593.337,3564.873,3543.781,3559.854,3606.405,3640.702,3704.007,3698.723,3664.697,3700.668
3rd_grade,2893.01,2894.524,3296.720,3361.362,3436.978,3439.042,3444.740,3523.762,3596.887,3695.643,...,3653.314,3622.790,3611.041,3580.462,3586.107,3627.487,3658.869,3707.772,3707.038,3685.833
4th_grade,3107.13,2771.015,3248.065,3341.737,3361.110,3425.951,3430.583,3453.769,3507.457,3592.153,...,3694.898,3668.517,3619.089,3611.638,3577.506,3585.891,3624.408,3646.896,3700.530,3711.341
5th_grade,3129.86,2776.402,3197.343,3325.427,3350.453,3371.884,3437.943,3452.853,3457.541,3520.104,...,3726.835,3711.466,3684.539,3635.181,3632.829,3601.796,3600.120,3629.079,3652.134,3717.521
6th_grade,3037.6,2788.817,3110.172,3302.671,3355.633,3381.439,3395.307,3493.631,3492.490,3496.812,...,3768.809,3787.997,3771.934,3735.281,3670.135,3660.368,3628.453,3613.521,3644.057,3682.052


This table shows the **Percent Distribution in Enrollment in Schools**.

In [24]:
# Second table
df2 = clean_table.iloc[20:]
df2.index.name = None
df2

,fall_1980,fall_1985,fall_1990,fall_1992,fall_1993,fall_1994,fall_1995,fall_1996,fall_1997,fall_1998,...,fall_2001,fall_2002,fall__2003,fall__2004,fall__2005,fall__2006,fall__2007,fall__2008,fall__2009,fall__2010
all_grades,100,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
elementary,67.6327,68.576609,72.485003,72.591394,72.476493,72.307877,72.118564,71.829653,71.695740,71.647501,...,71.186471,70.801287,70.458569,70.042503,69.642975,69.419378,69.392620,69.593354,69.709432,69.970906
prekindergarten,0.235173,0.383885,0.735913,1.178295,1.254195,1.365937,1.420248,1.469760,1.506973,1.565804,...,1.814668,1.898756,1.956417,2.029740,2.110357,2.198869,2.193285,2.394181,2.477382,2.584014
kindergarten,6.34361,7.714155,8.021822,7.735920,7.769480,7.808117,7.886238,7.744315,7.595332,7.397891,...,7.088354,7.126764,7.217273,7.262056,7.369534,7.362750,7.322442,7.388019,7.451171,7.440948
1st_grade,7.08085,8.215865,8.490334,8.270514,8.119358,8.146138,8.186583,8.266462,8.139706,8.008748,...,7.581135,7.458715,7.442301,7.506855,7.514979,7.606367,7.607795,7.527539,7.554090,7.586164
2nd_grade,6.84874,7.460296,8.073076,8.012671,7.889680,7.798033,7.821107,7.892992,7.997442,7.910561,...,7.537646,7.398598,7.300711,7.295461,7.343032,7.382419,7.514341,7.507724,7.424279,7.478487
3rd_grade,7.07726,7.342415,7.998509,7.849374,7.907476,7.796251,7.682210,7.725677,7.797808,7.941030,...,7.663459,7.518800,7.439277,7.337694,7.301703,7.355622,7.422769,7.526091,7.510057,7.448508
4th_grade,7.60107,7.029115,7.880462,7.803546,7.732926,7.766574,7.650638,7.572221,7.603930,7.718655,...,7.750688,7.613703,7.455857,7.401585,7.284190,7.271276,7.352858,7.402524,7.496873,7.500055
5th_grade,7.65669,7.042780,7.757400,7.765460,7.708408,7.644005,7.667052,7.570212,7.495716,7.563840,...,7.817682,7.702840,7.590694,7.449834,7.396834,7.303527,7.303585,7.366359,7.398828,7.512544
6th_grade,7.43099,7.074273,7.545906,7.712320,7.720326,7.665666,7.571968,7.659616,7.571483,7.513791,...,7.905729,7.861674,7.770740,7.654976,7.472793,7.422296,7.361064,7.334779,7.382465,7.440867
